In [1]:
!unzip India-covid-data.zip

Archive:  India-covid-data.zip
  inflating: Latest Covid-19 India Status.csv  


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [6]:
df = spark.read.csv('/content/Latest Covid-19 India Status.csv', inferSchema=True,
                    header=True)
df.limit(5)

State/UTs,Total Cases,Active,Discharged,Deaths,Active Ratio (%),Discharge Ratio (%),Death Ratio (%)
Andaman and Nicobar,7600,13,7458,129,0.17,98.13,1.7
Andhra Pradesh,2039529,14388,2011063,14078,0.71,98.6,0.69
Arunachal Pradesh,54126,410,53444,272,0.76,98.74,0.5
Assam,598423,4984,587632,5807,0.83,98.2,0.97
Bihar,725901,69,716173,9659,0.01,98.66,1.33


In [7]:
# Change columns name
from pyspark.sql import functions as F
renamed_df = df.select([F.col(col).alias(col.replace(' ', '_')) for col in df.columns])
renamed_df.limit(5)

State/UTs,Total_Cases,Active,Discharged,Deaths,Active_Ratio_(%),Discharge_Ratio_(%),Death_Ratio_(%)
Andaman and Nicobar,7600,13,7458,129,0.17,98.13,1.7
Andhra Pradesh,2039529,14388,2011063,14078,0.71,98.6,0.69
Arunachal Pradesh,54126,410,53444,272,0.76,98.74,0.5
Assam,598423,4984,587632,5807,0.83,98.2,0.97
Bihar,725901,69,716173,9659,0.01,98.66,1.33


In [8]:
renamed_df = renamed_df.withColumnRenamed('State/UTs', 'State_UTs')
renamed_df = renamed_df.withColumnRenamed('Active_Ratio_(%)', 'Active_Ratio')
renamed_df = renamed_df.withColumnRenamed('Discharge_Ratio_(%)', 'Discharge_Ratio')
renamed_df = renamed_df.withColumnRenamed('Death_Ratio_(%)', 'Death_Ratio')
renamed_df.limit(5)

State_UTs,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharge_Ratio,Death_Ratio
Andaman and Nicobar,7600,13,7458,129,0.17,98.13,1.7
Andhra Pradesh,2039529,14388,2011063,14078,0.71,98.6,0.69
Arunachal Pradesh,54126,410,53444,272,0.76,98.74,0.5
Assam,598423,4984,587632,5807,0.83,98.2,0.97
Bihar,725901,69,716173,9659,0.01,98.66,1.33


In [9]:
# Create Temp View to easier query
renamed_df.createOrReplaceTempView('Data')

In [10]:
spark.sql('Select * from Data').limit(5)

State_UTs,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharge_Ratio,Death_Ratio
Andaman and Nicobar,7600,13,7458,129,0.17,98.13,1.7
Andhra Pradesh,2039529,14388,2011063,14078,0.71,98.6,0.69
Arunachal Pradesh,54126,410,53444,272,0.76,98.74,0.5
Assam,598423,4984,587632,5807,0.83,98.2,0.97
Bihar,725901,69,716173,9659,0.01,98.66,1.33


In [11]:
spark.sql('select count(State_UTs) from Data')

count(State_UTs)
36


In [12]:
# 5 worst States by number of Active people
spark.sql('select * from Data order by Active desc limit 5')

State_UTs,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharge_Ratio,Death_Ratio
Kerala,4524158,167578,4332897,23683,3.7,95.77,0.52
Maharashtra,6524498,45229,6340723,138546,0.69,97.18,2.12
Tamil Nadu,2647041,16984,2594697,35360,0.64,98.02,1.34
Mizoram,81460,15140,66057,263,18.59,81.09,0.32
Andhra Pradesh,2039529,14388,2011063,14078,0.71,98.6,0.69


In [13]:
spark.sql('select * from Data order by Deaths desc limit 5')

State_UTs,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharge_Ratio,Death_Ratio
Maharashtra,6524498,45229,6340723,138546,0.69,97.18,2.12
Karnataka,2968543,14386,2916530,37627,0.48,98.25,1.27
Tamil Nadu,2647041,16984,2594697,35360,0.64,98.02,1.34
Delhi,1438517,379,1413053,25085,0.03,98.23,1.74
Kerala,4524158,167578,4332897,23683,3.7,95.77,0.52


In [14]:
spark.sql('select sum(Total_Cases), sum(Deaths) from Data')

sum(Total_Cases),sum(Deaths)
33504534,445385


In [15]:
spark.sql('select * from Data order by Active asc limit 5')

State_UTs,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharge_Ratio,Death_Ratio
Dadra and Nagar H...,10670,0,10666,4,0.0,99.96,0.04
Lakshadweep,10359,8,10300,51,0.08,99.43,0.49
Andaman and Nicobar,7600,13,7458,129,0.17,98.13,1.7
Chandigarh,65188,41,64329,818,0.06,98.68,1.25
Jharkhand,348125,55,342937,5133,0.02,98.51,1.47


In [16]:
import pandas as pd

pd_df = renamed_df.toPandas()
pd_df.head()

,State_UTs,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharge_Ratio,Death_Ratio
0,Andaman and Nicobar,7600,13,7458,129,0.17,98.13,1.70
1,Andhra Pradesh,2039529,14388,2011063,14078,0.71,98.60,0.69
2,Arunachal Pradesh,54126,410,53444,272,0.76,98.74,0.50
3,Assam,598423,4984,587632,5807,0.83,98.20,0.97
4,Bihar,725901,69,716173,9659,0.01,98.66,1.33


In [17]:
pd_df.describe()

,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharge_Ratio,Death_Ratio
count,3.600000e+01,36.000000,3.600000e+01,36.000000,36.000000,36.000000,36.000000
mean,9.306815e+05,8599.305556,9.097104e+05,12371.805556,1.111389,97.620278,1.267222
std,1.376378e+06,28560.135752,1.336378e+06,23784.824599,3.101292,2.982404,0.568057
min,7.600000e+03,0.000000,7.458000e+03,4.000000,0.000000,81.090000,0.040000
25%,8.101175e+04,136.750000,7.380750e+04,815.750000,0.037500,97.637500,0.955000
50%,4.732740e+05,438.500000,4.637270e+05,5470.000000,0.470000,98.280000,1.300000
75%,1.009009e+06,4956.250000,9.953420e+05,13691.750000,0.742500,98.660000,1.575000
max,6.524498e+06,167578.000000,6.340723e+06,138546.000000,18.590000,99.960000,2.740000


In [18]:
pd_df.corr()

,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharge_Ratio,Death_Ratio
Total_Cases,1.000000,0.661820,0.999934,0.890703,-0.042371,0.029231,0.074894
Active,0.661820,1.000000,0.654236,0.338358,0.208015,-0.179319,-0.197351
Discharged,0.999934,0.654236,1.000000,0.892334,-0.046605,0.033375,0.076274
Deaths,0.890703,0.338358,0.892334,1.000000,-0.083180,0.031644,0.285375
Active_Ratio,-0.042371,0.208015,-0.046605,-0.083180,1.000000,-0.983307,-0.296941
Discharge_Ratio,0.029231,-0.179319,0.033375,0.031644,-0.983307,1.000000,0.118244
Death_Ratio,0.074894,-0.197351,0.076274,0.285375,-0.296941,0.118244,1.000000


In [21]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=2, subplot_titles=['Total_Cases', 'Deaths', 'Active', 'Discharged'])

total_cases = go.Histogram(x=pd_df['Total_Cases'], nbinsx=20, name='Total Cases')
deaths = go.Histogram(x=pd_df['Deaths'], nbinsx=20, name='Deaths')
active = go.Histogram(x=pd_df['Active'], nbinsx=20, name='Active')
discharged = go.Histogram(x=pd_df['Discharged'], nbinsx=20, name='Discharged')

fig.add_trace(total_cases, 1, 1)
fig.add_trace(deaths, 1, 2)
fig.add_trace(active, 2, 1)
fig.add_trace(discharged, 2, 2)

fig.update_layout(showlegend=False)
fig

In [23]:
fig = make_subplots(rows=2, cols=2, subplot_titles=['Total_Cases', 'Deaths', 'Active', 'Discharged'])

total_cases = go.Box(x=pd_df['Total_Cases'], name='Total Cases', text=pd_df['State_UTs'])
deaths = go.Box(x=pd_df['Deaths'], name='Deaths', text=pd_df['State_UTs'])
active = go.Box(x=pd_df['Active'], name='Active', text=pd_df['State_UTs'])
discharged = go.Box(x=pd_df['Discharged'], name='Discharged', text=pd_df['State_UTs'])

fig.add_trace(total_cases, 1, 1)
fig.add_trace(deaths, 1, 2)
fig.add_trace(active, 2, 1)
fig.add_trace(discharged, 2, 2)

fig.update_layout(showlegend=False)
fig

In [24]:
fig = make_subplots(rows=2, cols=2, subplot_titles=['Total_Cases', 'Deaths', 'Active', 'Discharged'])

total_cases = go.Bar(y=pd_df['Total_Cases'], name='Total Cases', hovertext=pd_df['State_UTs'])
deaths = go.Bar(y=pd_df['Deaths'], name='Deaths', hovertext=pd_df['State_UTs'])
active = go.Bar(y=pd_df['Active'], name='Active', hovertext=pd_df['State_UTs'])
discharged = go.Bar(y=pd_df['Discharged'], name='Discharged', hovertext=pd_df['State_UTs'])

fig.add_trace(total_cases, 1, 1)
fig.add_trace(deaths, 1, 2)
fig.add_trace(active, 2, 1)
fig.add_trace(discharged, 2, 2)

fig.update_layout(showlegend=False)
fig

In [28]:
fig = go.Figure([go.Bar(y=pd_df['Total_Cases'], name='Total Cases', hovertext=pd_df['State_UTs'], x=pd_df['State_UTs']),
                 go.Bar(y=pd_df['Discharged'], name='Discharged', hovertext=pd_df['State_UTs'], x=pd_df['State_UTs'])])

fig.update_layout(barmode='group')
fig.update_layout(title='Total & Discharged cases')
fig.show()

In [38]:
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=['Total_Cases', 'Deaths', 'Active', 'Discharged'],
                    specs=[[{'type':'domain'}, {'type':'domain'}],
                           [{'type':'domain'}, {'type':'domain'}]])

total_cases = go.Pie(values=pd_df['Total_Cases'], name='Total Cases', labels=pd_df['State_UTs'])
deaths = go.Pie(values=pd_df['Deaths'], name='Deaths', labels=pd_df['State_UTs'])
active = go.Pie(values=pd_df['Active'], name='Active', labels=pd_df['State_UTs'])
discharged = go.Pie(values=pd_df['Discharged'], name='Discharged', labels=pd_df['State_UTs'])

fig.add_trace(total_cases, 1, 1)
fig.add_trace(deaths, 1, 2)
fig.add_trace(active, 2, 1)
fig.add_trace(discharged, 2, 2)

fig.update_traces(hoverinfo='percent+label')
fig.update_layout(showlegend=False)

fig.update_traces(textposition='inside')

fig = go.Figure(fig)
fig.show()

In [41]:
fig = go.Figure([go.Scatter(x=pd_df['Total_Cases'], y=pd_df['Deaths'], mode='markers', text=pd_df['State_UTs'])])
fig.update_layout(title='Deaths vs Total Cases', xaxis_title='Total Cases', yaxis_title='Deaths')
fig.show()

In [42]:
X = pd_df['Total_Cases'].to_numpy()
y = pd_df['Deaths'].to_numpy()

X.shape, y.shape

((36,), (36,))

In [43]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X.reshape((len(X), 1)), y)
(model.intercept_, model.coef_[0])

(-1953.2560563403458, 0.015392012854984117)

In [44]:
model.score(X.reshape((len(X), 1)), y)

0.7933522500131218

In [45]:
predictions = model.predict(X.reshape((len(X), 1)))
predictions.shape

(36,)

In [46]:
fig = go.Figure([go.Scatter(x=pd_df['Total_Cases'], y=pd_df['Deaths'], mode='markers', text=pd_df['State_UTs'], name='Actual Deaths'),
                 go.Scatter(x=pd_df['Total_Cases'], y=predictions, name='Predicted Deaths')])
fig.update_layout(title='Deaths vs Total Cases', xaxis_title='Total Cases', yaxis_title='Deaths')
fig.show()